In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

## Diseño de redes adaptadores tipo L

A continuación se calculan los distintos valores de susceptancia y reactancia para distintos valores de $Z_L$.
Si se cumple que $R_L > Z_o$ se utilizan las siguientes fórmulas:
$B = \frac{X_L \pm \sqrt{\frac{R_L}{Z_o}}\cdot \sqrt{R_L^2 + X_L^2Z_oR_L}}{R_L^2 + X_L^2}$
$X = \frac{1}{B} + \frac{X_LZ_o}{R_L} + \frac{Z_o}{BR_L}$
Si se cumple que $R_L < Z_o$ se utilizan las siguientes fórmulas:
$X = \pm \sqrt{R_L(Z_o-R_L)} - X_L$
$B = \pm \sqrt{\frac{Z_o - R_L}{R_L}}\cdot\frac{1}{Z_o}$

Datos del Problema:

In [2]:
Zo = 50 #Ω
ZL = [1.4+2J, 0.2+0.3J, 0.5+0.9J, 1.6-0.3J] #Impedancia normalizadas
for i in range(len(ZL)):
    ZL[i] = ZL[i]*Zo
print("Impedancia Característica (Zo) = {}".format(Zo))
print("Impedancias ZL: {}".format(ZL))

Impedancia Característica (Zo) = 50
Impedancias ZL: [(70+100j), (10+15j), (25+45j), (80-15j)]


Se nombra al caso $R_L > Z_o$ CASO I, entonces se definen:

In [3]:
B_1 = lambda RL, XL: [(XL+(np.sqrt(RL/Zo)*np.sqrt(RL**2 + XL**2 - (Zo*RL))))/(RL**2 + XL**2), (XL-(np.sqrt(RL/Zo)*np.sqrt(RL**2 + XL**2 - (Zo*RL))))/(RL**2 + XL**2)]
X_1 = lambda RL, XL, B: [(1/B[0]) + ((XL*Zo)/RL) - (Zo/(RL*B[0])), (1/B[1]) + ((XL*Zo)/RL) - (Zo/(RL*B[1]))]

Se al nombre al caso $R_L < Z_o$ CASO II, entonces se definen:

In [4]:
X_2 = lambda RL, XL: [np.sqrt(RL*(Zo-RL))-XL, -np.sqrt(RL*(Zo-RL))-XL]
B_2 = lambda RL, XL: [np.sqrt((Zo-RL)/RL)/Zo, -np.sqrt((Zo-RL)/RL)/Zo]

In [5]:
results = []
caso = []
for zl in ZL:
    if (zl.real > Zo):
        results.append([B_1(zl.real, zl.imag),X_1(zl.real, zl.imag, B_1(zl.real, zl.imag)), [zl, "ZL > Zo"]])
        caso.append("ZL > Zo")
    else:
        results.append([B_2(zl.real, zl.imag), X_2(zl.real, zl.imag), [zl, "ZL < Zo"]])
        caso.append("ZL < Zo")
for i in range(len(ZL)):
    print("ZL = {}, caso: ".format(ZL[i]) + caso[i] +", opcion 1: B = {}, X = {} opcion 2: B = {}, X = {} ".format(round(results[i][0][0],5), round(results[i][1][0],5), round(results[i][0][1],5), round(results[i][1][1]),5))

ZL = (70+100j), caso: ZL > Zo, opcion 1: B = 0.01519, X = 90.23778 opcion 2: B = -0.00177, X = -90 
ZL = (10+15j), caso: ZL < Zo, opcion 1: B = 0.04, X = 5.0 opcion 2: B = -0.04, X = -35 
ZL = (25+45j), caso: ZL < Zo, opcion 1: B = 0.02, X = -20.0 opcion 2: B = -0.02, X = -70 
ZL = (80-15j), caso: ZL > Zo, opcion 1: B = 0.00752, X = 40.50463 opcion 2: B = -0.01205, X = -41 


Una vez calculadas las susceptancias y reactancias, se calculan los componentes con la siguiente regla:
\begin{align*}
    &X > 0: \text{Inductor Serie} \rightarrow L = \frac{X}{\omega}&
    & X < 0: \text{Capacitor Serie} \rightarrow C = \frac{1}{-\omega X} \\
    &B > 0: \text{Capacitor Paralelo} \rightarrow C = \frac{B}{\omega}&
    &B < 0: \text{Capacitor Paralelo} \rightarrow C = \frac{1}{-B \omega}
\end{align*}

In [6]:
f = 1*1e9 #Hz
W = 2*math.pi*f

In [7]:
for i in results:
    B_comp = []
    for B in i[0]:
        if(B > 0):
            B_comp.append(["Capacitor Paralelo",B/W])
        else:
            B_comp.append(["Inductor Paralelo",-1/(B*W)])
    X_comp = []
    for X in i[1]:
        if(X > 0):
            X_comp.append(["Inductor Serie",X/W])
        else:
            X_comp.append(["Capacitor Serie",-1/(X*W)])
    i.append([B_comp, X_comp])
print(results[0])

[[0.01519012708582752, -0.0017673082938812106], [90.23778112773574, -90.23778112773579], [(70+100j), 'ZL > Zo'], [[['Capacitor Paralelo', 2.417583811903537e-12], ['Inductor Paralelo', 9.005499699340681e-08]], [['Inductor Serie', 1.436178892012369e-08], ['Capacitor Serie', 1.7637284638748385e-12]]]]


Para cada valor de $Z_L$ se obtiene lo siguiente:

In [8]:
for i in results:
    print("Para el caso zl = {}, caso {}, se tienen las dos siguientes soluciones: ".format(i[2][0], i[2][1]))
    #Imprimo Solución 1
    print("Solución 1: ")
    if(i[3][0][0][0] == "Capacitor Paralelo"):
        print("C_p = {} F".format(i[3][0][0][1]))
    else:
        print("L_p = {} H".format(i[3][0][0][1]))
    if(i[3][1][0][0] == "Capacitor Serie"):
        print("C_s = {} F".format(i[3][1][0][1],2))
    elif (i[3][1][0][0] == "Inductor Serie"):
        print("L_s = {} H".format(i[3][1][0][1],2))
      #Imprimo Solución 2
    print("Solución 2: ")
    if(i[3][0][1][0] == "Capacitor Paralelo"):
        print("C_p = {} F".format(i[3][0][1][1],2))
    else:
        print("L_p = {} H".format(i[3][0][1][1],2))
    if(i[3][1][1][0] == "Capacitor Serie"):
        print("C_s = {} F".format(i[3][1][1][1],2))
    elif (i[3][1][1][0] == "Inductor Serie"):
        print("L_s = {} H".format(i[3][1][1][1],2))

Para el caso zl = (70+100j), caso ZL > Zo, se tienen las dos siguientes soluciones: 
Solución 1: 
C_p = 2.417583811903537e-12 F
L_s = 1.436178892012369e-08 H
Solución 2: 
L_p = 9.005499699340681e-08 H
C_s = 1.7637284638748385e-12 F
Para el caso zl = (10+15j), caso ZL < Zo, se tienen las dos siguientes soluciones: 
Solución 1: 
C_p = 6.3661977236758135e-12 F
L_s = 7.957747154594766e-10 H
Solución 2: 
L_p = 3.9788735772973836e-09 H
C_s = 4.5472840883398665e-12 F
Para el caso zl = (25+45j), caso ZL < Zo, se tienen las dos siguientes soluciones: 
Solución 1: 
C_p = 3.1830988618379067e-12 F
C_s = 7.957747154594766e-12 F
Solución 2: 
L_p = 7.957747154594767e-09 H
C_s = 2.2736420441699333e-12 F
Para el caso zl = (80-15j), caso ZL > Zo, se tienen las dos siguientes soluciones: 
Solución 1: 
C_p = 1.1965426451480606e-12 F
L_s = 6.446511981552704e-09 H
Solución 2: 
L_p = 1.3211825040176498e-08 H
C_s = 3.929302541136888e-12 F
